Notebook purpose

- Replicate analysis of davenport2020spending

In [99]:
import os
import sys
import numpy as np
import pandas as pd
sys.path.append('/Users/fgu/dev/projects/entropy')
import entropy.helpers.aws as aws
import entropy.data.cleaners as cl

pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 120)
pd.set_option('max_colwidth', None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
d = aws.S3BucketManager("")
d.list()

['3di-data-experian',
 '3di-data-fable',
 '3di-data-huq',
 '3di-data-mdb',
 '3di-data-ons',
 '3di-project-entropy',
 '3di-project-habits',
 '3di-project-te']

In [415]:
df = aws.s3read_parquet('s3://3di-data-mdb/raw/mdb_000.parquet')


In [426]:
df = aws.s3read_parquet('s3://3di-project-entropy/entropy_000.parquet')
df.head(2)

,id,date,user_id,amount,desc,merchant,tag_up,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_recorded_balance,merchant_business_line,tag,tag_auto,tag_manual
0,688283,2012-01-17,777,42.410000,"9572 16jan12 , esso altrincham , sstn , altrincham gb - pos",esso,fuel,False,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-06-25 22:55:00,natwest bank,current,2014-07-18,2017-08-15,True,2378.280029,esso,fuel,fuel,NaN
1,248375,2012-07-23,777,39.389999,"9572 20jul12 , sacat marks & , spencer , gemini gb , cash back 30.00 - pos",NaN,groceries,False,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-06-25 22:55:00,natwest bank,current,2014-07-18,2017-10-23,True,2378.280029,personal,groceries,cash,groceries
